In [335]:
import pandas as pd
import numpy as np
import random
import copy
import math
from collections import defaultdict

randomlist = []
df_demand = {}
random.seed(3) # Use the same random seed
for i in range(0,50):   # 20 depots
    n = random.randint(50,150) # Depot Capacity
    randomlist.append(n)

for i in range(50):
    df_demand['D'+str(i+1)] = randomlist[i]

sum = 0
for (k,v) in df_demand.items():
    sum = sum + v

df_supply = {}
for i in range(sum):
    df_supply[i] = 1.0

np.random.seed(3) # Use the same random seed
df_dkm = pd.DataFrame(np.random.uniform(0.5,20, size=(sum,50))).astype(float)

random.seed(3) # Use the same random seed
age_b = []
for i in range(sum):
    n = random.randint(1,3) #Only to generate integer values inclusive of 1 & 3
    age_b.append(n)

age = {}
for i in range(len(age_b)):
    age[i] = age_b[i]

kpl = []
co2 = []
random.seed(3) # Use the same random seed
for (k,v) in age.items():
    if (v==1):
        n = random.uniform(5.1,6)
        m = round(random.uniform(515, 524), 2)
        kpl.append(n)
        co2.append(m)
    if (v==2):
        n = random.uniform(4.1,5)
        m = round(random.uniform(525, 534), 2)
        kpl.append(n)
        co2.append(m)
    if (v==3):
        n = random.uniform(3.1,4)
        m = round(random.uniform(535, 540), 2)
        kpl.append(n)
        co2.append(m)

fc_kpl = {}
for i in range(sum):
    fc_kpl[i] =(111/ kpl[i])

co2_c = {}
for i in range(sum):
    co2_c[i] =(0.00118642*co2[i])

random.seed(3) # Use the same random seed
doc = {}
for i in range(0,50):
    n = random.uniform(50,100)
    doc[i] = n

df_tdkom = pd.DataFrame(np.zeros(df_dkm.shape))
for i in range(len(df_dkm.columns)):
    for j in range(len(df_dkm.index)):
        df_tdkom[i][j] = df_dkm[i][j]*fc_kpl[j] + df_dkm[i][j]*co2_c[j] + doc[i]
#df_tdkom

df_tdkom.columns = [("D"+str(i)) for i in range(1,51)]

dk = df_tdkom.transpose()
costs_t = dk.to_dict()
colsn = sorted(df_demand.keys())

# print(supply['S1'])
# break

costs1=copy.deepcopy(costs_t)
costs2=copy.deepcopy(costs_t)
costs3=copy.deepcopy(costs_t)
for i in df_supply:
    mi=min(costs_t[i].values())
    # print(costs[i])
    # print(mi)
    for j in costs2[i]:
        costs2[i][j]-=mi
# print(costs2)
for i in df_demand :
    mi=10000
    for j in df_supply:
        if costs_t[j][i]<mi :
            mi=costs_t[j][i]
    for j in df_supply:
        costs3[j][i]=costs3[j][i]-mi
# print(costs3)

for i in df_demand:
    for j in df_supply:
        costs_t[j][i]= costs2[j][i]+costs3[j][i]
# print(costs_t)


res = dict((k, defaultdict(int)) for k in costs_t)

g = {}
for x in df_supply:
    # print(x)
    # print(costs_t[x])
    g[x] = sorted(costs_t[x].keys(), key=lambda g: costs_t[x][g])
for x in df_demand:
    g[x] = sorted(costs_t.keys(), key=lambda g: costs_t[g][x])

while g:
    d = {}
    # print(df_supply,dic)
    for x in df_demand:
        d[x] = ((costs_t[g[x][1]][x] - costs_t[g[x][0]][x])/df_demand[x])*costs_t[g[x][0]][x] if len(g[x]) > 1 else (costs_t[g[x][0]][x]/df_demand[x])*costs_t[g[x][0]][x]
    s = {}
    for x in df_supply:
        s[x] = ((costs_t[x][g[x][1]] - costs_t[x][g[x][0]])/df_supply[x])*costs_t[x][g[x][0]]  if len(g[x]) > 1 else (costs_t[x][g[x][0]]/df_supply[x])*costs_t[x][g[x][0]]

    f = max(d, key=lambda n: d[n])
    t = max(s, key=lambda n: s[n])


    t, f = (f, g[f][0]) if d[f] >= s[t] else (g[t][0], t)


    v = min(df_supply[f], df_demand[t])
    # print(f,t)
    # print(v)
    res[f][t] += v
    df_demand[t] -= v

    if df_demand[t] == 0:
        for k, n in df_supply.items():
            if n != 0:
                g[k].remove(t)
        del g[t]
        del df_demand[t]
    df_supply[f] -= v
    if df_supply[f] == 0:
        for k, n in df_demand.items():
            if n != 0:
                g[k].remove(f)
        del g[f]
        del df_supply[f]

In [336]:
# print("G",g)
cost = 0
# cols = sorted(df_demand.keys())
# print(costs)
for g in sorted(costs1):
    # print (g, " ",)
    # print("S")
    for n in colsn:
        y = res[g][n]
        # print("YESS",y)
        if y != 0:
            pass
            # print (y,)
        cost += y * costs1[g][n]
        # print ("  ",)
    # print(" ")

In [337]:
df_tdkom

,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,...,D41,D42,D43,D44,D45,D46,D47,D48,D49,D50
0,303.619469,384.915088,201.236597,305.156280,466.482457,439.876037,114.072953,189.529915,95.301909,257.315901,...,279.494381,279.203521,204.984608,198.162534,278.038440,440.897819,324.134299,208.882386,206.530583,296.601399
1,210.453337,223.174417,415.483961,152.766328,408.694612,302.624588,522.153399,588.481931,523.793736,521.312931,...,581.749065,397.065643,639.250909,560.377803,516.406449,673.422157,491.448484,331.022164,451.151474,237.959480
2,193.238176,530.454737,557.495093,682.102377,610.120121,396.679130,120.295014,402.264547,638.232450,552.331550,...,635.040061,262.751218,543.873762,175.856759,510.081299,592.365104,521.683121,311.361015,533.071278,194.814904
3,209.738853,271.113934,461.705509,158.319558,475.762318,412.405991,484.984282,263.181410,163.389529,177.656083,...,334.509422,151.030116,518.414297,295.174978,447.573213,112.705123,287.288388,117.544570,475.481759,300.535047
4,110.094321,129.473586,357.309985,131.982322,188.439674,289.673927,318.486515,403.465515,393.594835,265.598713,...,401.553559,397.871588,115.959096,291.680631,90.665347,208.561932,441.943982,123.863291,243.122948,613.932650
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5249,464.866447,281.301856,228.439372,253.863290,331.120466,333.328299,473.491974,592.087707,237.323497,133.462349,...,137.475499,155.624082,122.840035,434.394781,279.552594,583.796765,313.727761,364.473001,276.353652,173.202009
5250,262.839328,319.515139,210.856301,364.269335,306.908819,541.390302,121.866838,176.842275,336.200102,317.503997,...,545.967630,113.595357,464.932771,584.722417,536.882736,570.421051,304.344550,409.734726,251.454345,284.714050
5251,594.005594,322.089393,421.537252,332.772250,520.562948,470.461469,254.312476,419.837267,483.746565,160.924641,...,374.939439,270.075086,154.779034,606.354589,512.258922,445.104532,576.275337,586.763837,480.518286,238.555045
5252,282.649153,165.701066,117.567912,447.440383,123.063631,343.162210,393.896469,343.860431,358.636405,445.841013,...,193.658780,87.750903,108.877935,138.636327,124.172881,368.419979,191.546127,464.529372,214.383900,337.859035


In [338]:
cost

551901.2968350239